# 1.6 套利定价理论 (Arbitrage Pricing Theory, APT)

## 概念介绍

套利定价理论(APT)是由Stephen Ross于1976年提出的一种多因子资产定价模型，是对资本资产定价模型(CAPM)的扩展。与CAPM只考虑市场风险不同，APT认为资产回报率受多种系统性风险因子影响。

### APT模型公式

套利定价理论的基本方程：

$$E(R_i) = R_f + \beta_{i1}RP_1 + \beta_{i2}RP_2 + ... + \beta_{in}RP_n$$

其中：
- $E(R_i)$ 是资产$i$的期望收益率
- $R_f$ 是无风险收益率
- $\beta_{ij}$ 是资产$i$对因子$j$的敏感度
- $RP_j$ 是因子$j$的风险溢价

### APT与CAPM的区别

- **多因子模型**：APT允许多个因子影响资产收益率，而CAPM只考虑市场风险一个因子
- **更少的假设**：APT比CAPM有更少的限制性假设，更适合现实市场
- **因子选择灵活**：APT允许使用各种宏观经济因子，如GDP增长率、通货膨胀率、利率等

在这个笔记本中，我们将使用宏观经济因子构建APT模型，分析这些因子如何影响股票收益率。

## 1.1 安装必要的库

首先安装需要用到的Python库

In [ ]:
# 安装必要的库
!pip install pandas numpy matplotlib statsmodels seaborn scikit-learn

## 1.2 从Github上读取数据

In [ ]:
!wget -O AAPL_data.csv "https://github.com/Newzil-git/Financial-Risk-Management/blob/update-repo/FR%20Code/Part%201/AAPL_data.csv"
!wget -O SPY_data.csv "https://github.com/Newzil-git/Financial-Risk-Management/blob/update-repo/FR%20Code/Part%201/SPY_data.csv"
!wget -O MSFT_data.csv ""
!wget -O GOOGL_data.csv ""
!wget -O risk_free_rate.csv ""
!wget -O unemployment_rate.csv ""
!wget -O inflation_rate.csv ""
!wget -O gdp_data.csv ""

# 设置数据文件路径
file_path_msft = "MSFT_data.csv"
file_path_aapl = "AAPL_data.csv"
file_path_googl = "GOOGL_data.csv"
file_path_market = "SPY_data.csv"  # 市场数据
file_path_rf =  "risk_free_rate.csv"  # 无风险利率
file_path_unrate =  "unemployment_rate.csv"  # 失业率
file_path_cpi = "inflation_rate.csv"  # 通胀率
file_path_gdp =  "gdp_data.csv"  # GDP数据

## 1.3 导入必要的库

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
from sklearn.preprocessing import StandardScaler

# 设置中文显示
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False    # 用来正常显示负号

## 1.4 数据加载和预处理

In [ ]:
# 读取股票数据和宏观经济数据
try:
    msft_data = pd.read_csv(file_path_msft)
    aapl_data = pd.read_csv(file_path_aapl)
    googl_data = pd.read_csv(file_path_googl)
    market_data = pd.read_csv(file_path_market)
    rf_data = pd.read_csv(file_path_rf)
    unrate_data = pd.read_csv(file_path_unrate)
    cpi_data = pd.read_csv(file_path_cpi)
    gdp_data = pd.read_csv(file_path_gdp)
    print("数据加载成功")
except Exception as e:
    print(f"数据加载失败: {e}")

In [ ]:
# 数据预览
print("MSFT数据预览:")
display(msft_data.head())
print("\n无风险利率数据预览:")
display(rf_data.head())
print("\n失业率数据预览:")
display(unrate_data.head())
print("\n通胀率数据预览:")
display(cpi_data.head())
print("\nGDP数据预览:")
display(gdp_data.head())

In [ ]:
# 统一列名（将 "Close" 重命名为 "Adj Close"，如果需要）
for df in [msft_data, aapl_data, googl_data, market_data]:
    if 'Close' in df.columns and 'Adj Close' not in df.columns:
        df.rename(columns={'Close': 'Adj Close'}, inplace=True)

# 转换日期格式
for df in [msft_data, aapl_data, googl_data, market_data, rf_data, unrate_data, cpi_data, gdp_data]:
    if 'Date' in df.columns:
        try:
            df['Date'] = pd.to_datetime(df['Date'])
        except Exception as e:
            print(f"日期转换错误: {e}")

In [ ]:
# 设置日期为索引
for df in [msft_data, aapl_data, googl_data, market_data, rf_data, unrate_data, cpi_data, gdp_data]:
    if 'Date' in df.columns:
        df.set_index('Date', inplace=True)

# 计算每日收益率
msft_returns = msft_data['Adj Close'].pct_change().dropna()
aapl_returns = aapl_data['Adj Close'].pct_change().dropna()
googl_returns = googl_data['Adj Close'].pct_change().dropna()
market_returns = market_data['Adj Close'].pct_change().dropna()

In [ ]:
# 合并股票收益率和宏观经济数据
# 首先创建一个包含股票收益率的DataFrame
returns_df = pd.DataFrame({
    'MSFT': msft_returns,
    'AAPL': aapl_returns,
    'GOOGL': googl_returns,
    'MKT': market_returns
})

# 将宏观经济因子添加到returns_df
# 注意：宏观经济数据通常频率较低（月度/季度），需要进行适当的频率转换
# 这里假设所有数据已经对齐到相同的频率

# 合并无风险利率
if 'GS10' in rf_data.columns:  # 假设10年期国债收益率的列名是'GS10'
    returns_df = returns_df.join(rf_data['GS10'], how='left')

# 合并失业率
if 'UNRATE' in unrate_data.columns:
    returns_df = returns_df.join(unrate_data['UNRATE'], how='left')

# 合并通胀率
if 'CPIAUCSL' in cpi_data.columns:
    returns_df = returns_df.join(cpi_data['CPIAUCSL'], how='left')

# 合并GDP
if 'GDP' in gdp_data.columns:
    returns_df = returns_df.join(gdp_data['GDP'], how='left')

# 重置索引，将日期作为列
returns_df = returns_df.reset_index()

# 处理缺失值（前向填充宏观经济数据）
macro_columns = ['GS10', 'UNRATE', 'CPIAUCSL', 'GDP']
for col in macro_columns:
    if col in returns_df.columns:
        returns_df[col] = returns_df[col].fillna(method='ffill')

# 删除仍然包含NaN的行
returns_df = returns_df.dropna()

In [ ]:
# 计算超额收益率（相对于无风险利率）
if 'GS10' in returns_df.columns:
    # 将年化无风险利率转换为日收益率
    returns_df['RF_daily'] = returns_df['GS10'] / 252
    
    # 计算超额收益率
    returns_df['MSFT_excess'] = returns_df['MSFT'] - returns_df['RF_daily']
    returns_df['AAPL_excess'] = returns_df['AAPL'] - returns_df['RF_daily']
    returns_df['GOOGL_excess'] = returns_df['GOOGL'] - returns_df['RF_daily']
    returns_df['MKT_excess'] = returns_df['MKT'] - returns_df['RF_daily']

# 基本统计信息
print("超额收益率统计信息:")
display(returns_df[['MSFT_excess', 'AAPL_excess', 'GOOGL_excess', 'MKT_excess']].describe())

print("\n宏观经济因子统计信息:")
macro_cols = [col for col in macro_columns if col in returns_df.columns]
display(returns_df[macro_cols].describe())

## 2. APT模型构建

在这一部分，我们将构建APT模型，分析宏观经济因子对股票超额收益率的影响。

In [ ]:
# 为回归分析准备自变量（宏观经济因子）
X_columns = [col for col in ['GS10', 'UNRATE', 'CPIAUCSL', 'GDP'] if col in returns_df.columns]
X = returns_df[X_columns]

# 添加常数项
X = sm.add_constant(X)

In [ ]:
# 对MSFT进行回归分析
y_msft = returns_df['MSFT_excess']
model_msft = sm.OLS(y_msft, X).fit()
print("MSFT回归结果:")
print(model_msft.summary())

In [ ]:
# 对AAPL进行回归分析
y_aapl = returns_df['AAPL_excess']
model_aapl = sm.OLS(y_aapl, X).fit()
print("AAPL回归结果:")
print(model_aapl.summary())

In [ ]:
# 对GOOGL进行回归分析
y_googl = returns_df['GOOGL_excess']
model_googl = sm.OLS(y_googl, X).fit()
print("GOOGL回归结果:")
print(model_googl.summary())

## 2.1 因子敏感度分析

分析不同股票对宏观经济因子的敏感度（Beta系数）

In [ ]:
# 提取每个股票的系数（Beta）
betas = pd.DataFrame({
    'MSFT': model_msft.params,
    'AAPL': model_aapl.params,
    'GOOGL': model_googl.params
})

# 提取显著性水平（p值）
p_values = pd.DataFrame({
    'MSFT': model_msft.pvalues,
    'AAPL': model_aapl.pvalues,
    'GOOGL': model_googl.pvalues
})

# 创建一个包含Beta系数和p值的表格
beta_table = pd.DataFrame(index=betas.index)

for stock in ['MSFT', 'AAPL', 'GOOGL']:
    beta_table[f'{stock}_Beta'] = betas[stock]
    beta_table[f'{stock}_p'] = p_values[stock]
    # 标记显著性
    beta_table[f'{stock}_Sig'] = np.where(p_values[stock] < 0.05, '*', '')
    beta_table[f'{stock}_Sig'] = np.where(p_values[stock] < 0.01, '**', beta_table[f'{stock}_Sig'])
    beta_table[f'{stock}_Sig'] = np.where(p_values[stock] < 0.001, '***', beta_table[f'{stock}_Sig'])

display(beta_table)

In [ ]:
# 计算因子的贡献
# 假设因子的风险溢价等于因子的平均值
risk_premiums = returns_df[X_columns].mean()

# 计算每个因子对风险溢价的贡献
contributions = pd.DataFrame(index=X_columns)

for stock, model in zip(['MSFT', 'AAPL', 'GOOGL'], [model_msft, model_aapl, model_googl]):
    # 提取除常数项外的系数
    factor_betas = model.params[1:]
    
    # 计算每个因子的贡献
    contributions[stock] = factor_betas * risk_premiums

display(contributions)

## 2.2 可视化结果

In [ ]:
# 可视化Beta系数
plt.figure(figsize=(12, 8))

# 提取显著的Beta系数（p < 0.05）
significant_betas = pd.DataFrame(index=betas.index)
for stock in ['MSFT', 'AAPL', 'GOOGL']:
    significant_betas[stock] = np.where(p_values[stock] < 0.05, betas[stock], np.nan)

# 只考虑非常数项的因子
factor_betas = significant_betas.iloc[1:]

# 绘制条形图
ax = factor_betas.plot(kind='bar', width=0.8)

# 添加水平线表示零
plt.axhline(y=0, color='black', linestyle='-', alpha=0.3)

# 设置图表标题和标签
plt.title('各股票对宏观经济因子的敏感度系数（显著Beta）', fontsize=14)
plt.xlabel('宏观经济因子', fontsize=12)
plt.ylabel('Beta系数 (敏感度)', fontsize=12)
plt.legend(title='股票')
plt.grid(axis='y', linestyle='--', alpha=0.7)

# 显示图表
plt.tight_layout()
plt.show()

In [ ]:
# 可视化模型解释能力（R²）
r_squared = pd.DataFrame({
    'R²': [model_msft.rsquared, model_aapl.rsquared, model_googl.rsquared],
    'Adj R²': [model_msft.rsquared_adj, model_aapl.rsquared_adj, model_googl.rsquared_adj]
}, index=['MSFT', 'AAPL', 'GOOGL'])

plt.figure(figsize=(10, 6))
r_squared.plot(kind='bar', width=0.8)
plt.title('APT模型解释能力比较', fontsize=14)
plt.ylabel('R²值', fontsize=12)
plt.ylim(0, 1)  # R²范围是0到1
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.legend(title='指标')

# 在每个柱子上添加数值标签
for i, v in enumerate(r_squared['R²']):
    plt.text(i-0.15, v+0.02, f'{v:.3f}', rotation=0, fontsize=10)
for i, v in enumerate(r_squared['Adj R²']):
    plt.text(i+0.1, v+0.02, f'{v:.3f}', rotation=0, fontsize=10)

plt.tight_layout()
plt.show()

In [ ]:
# 相关性热图
corr_matrix = returns_df[['MSFT_excess', 'AAPL_excess', 'GOOGL_excess'] + X_columns].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1, center=0, 
            linewidths=0.5, cbar_kws={"shrink": 0.8})
plt.title('因子与股票超额收益率的相关性矩阵', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# 可视化预测值与实际值的比较
plt.figure(figsize=(10, 8))

# 为每只股票创建一个子图
for i, (stock, model, y) in enumerate(zip(['MSFT', 'AAPL', 'GOOGL'], [model_msft, model_aapl, model_googl], [y_msft, y_aapl, y_googl])):
    plt.subplot(2, 2, i+1)
    
    # 绘制散点图
    plt.scatter(y, model.predict(), alpha=0.5)
    
    # 添加45度线（完美预测）
    min_val = min(min(y), min(model.predict()))
    max_val = max(max(y), max(model.predict()))
    plt.plot([min_val, max_val], [min_val, max_val], 'r--')
    
    # 添加标题和标签
    plt.title(f'{stock} 预测值 vs 实际值', fontsize=12)
    plt.xlabel('实际超额收益率', fontsize=10)
    plt.ylabel('预测超额收益率', fontsize=10)
    plt.grid(True, linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

## 2.3 结果分析与解释

通过上述APT模型分析，我们可以得出以下结论：

1. **模型解释能力分析**：
   - 多因子APT模型对MSFT、AAPL和GOOGL超额收益率的解释能力（R²）分别为[模型R²值]
   - 调整后的R²值考虑了变量数量的影响，更准确地反映了模型的解释能力
   - 总体来看，宏观经济因子能够解释股票超额收益的一部分变动，但仍有较大部分由其他因素（如公司特质因素）驱动

2. **因子敏感度分析**：
   - 利率(GS10)：[利率对各股票的影响分析]
   - 失业率(UNRATE)：[失业率对各股票的影响分析]
   - 通胀率(CPIAUCSL)：[通胀率对各股票的影响分析] 
   - GDP：[GDP对各股票的影响分析]
   - 不同科技股对宏观经济因子的敏感度各不相同，反映了它们业务模式和风险特征的差异

3. **股票间对比**：
   - MSFT：[微软公司对宏观经济因子的敏感度特点]
   - AAPL：[苹果公司对宏观经济因子的敏感度特点] 
   - GOOGL：[谷歌公司对宏观经济因子的敏感度特点]
   - 三家科技巨头虽同属技术行业，但对宏观经济环境的反应存在一定差异
   
4. **投资策略建议**：
   - 基于APT模型结果，投资者可以根据对未来宏观经济因子的预期来调整投资组合
   - 例如，如果预期利率将上升，可以增持对利率变动敏感度较低或呈负相关的股票
   - 多元化投资不同对宏观因子敏感度的股票，可以降低组合的系统性风险

5. **模型局限性**：
   - APT模型假设线性关系，但实际市场中因子与收益率的关系可能更为复杂
   - 本分析使用历史数据，未来关系可能发生变化
   - 可能存在遗漏的重要因子，如行业特定因子、全球经济因子等
   - 定期更新模型并结合其他评估方法将有助于提高投资决策的准确性